# spark_sql_interactive

Interactively execute arbitrary SQL queries againts CSV and PARQUET files

In [13]:
import glob
import logging
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import boto3
import os
import shutil
import sys
import re

In [3]:
# name of resulting file (default: data_result.csv)
output_result_file = os.environ.get('output_result_file', 'data_result.csv')

# file name for CSV or PARQUET file - must end with .csv or .parquet (default: data.csv)
data_file = os.environ.get('data_file', 'data.csv')

# master url of spark master (default: local mode)
master = os.environ.get('master', "local[*]")

# data_dir temporal data storage for local execution (default: ../../data/)
data_dir = os.environ.get('data_dir', '../../data/')

# sql statement to execute, table name == df, example: select * from df
sql = os.environ.get('sql')

In [4]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)

In [5]:
sc = SparkContext.getOrCreate(SparkConf().setMaster(master))
spark = SparkSession.builder.getOrCreate()

22/10/13 16:44:28 WARN Utils: Your hostname, li-c7445bcc-23db-11b2-a85c-f854e76b2a39.ibm.com resolves to a loopback address: 127.0.0.1; using 192.168.126.176 instead (on interface wlp4s0)
22/10/13 16:44:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/13 16:44:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [15]:



session = boto3.session.Session(profile_name='profile_name')
sts_connection = session.client('sts')
response = sts_connection.assume_role(RoleArn='arn:aws:iam:::role/role_name', RoleSessionName='role_name',DurationSeconds=3600)
credentials = response['Credentials']

conf = pyspark.SparkConf()

conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.0')

sc = SparkContext(conf=conf)
sc._jsc.hadoopConfiguration().set('fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider')
sc._jsc.hadoopConfiguration().set('fs.s3a.access.key', credentials['AccessKeyId'])
sc._jsc.hadoopConfiguration().set('fs.s3a.secret.key', credentials['SecretAccessKey'])
sc._jsc.hadoopConfiguration().set('fs.s3a.session.token', credentials['SessionToken'])
url = str('s3a://data.csv')

l1 = sc.textFile(url).collect()
for each in l1:
    print(str(each))
    break


NoCredentialsError: Unable to locate credentials

In [6]:
sc.textFile("s3a://landsat-pds/scene_list.gz").count()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:275)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFo

In [ ]:
if '.csv' in data_file:
    df = spark.read.option('header', 'true').option("inferSchema", "true").csv(data_dir + data_file)
elif '.parquet' in data_file:
    df = spark.read.parquet.csv(data_dir + data_file)
else:
    raise Exception('Neither .csv nor .parquet in file name')

In [ ]:
df.createOrReplaceTempView('df')
df = spark.sql(sql)

In [ ]:
if os.path.exists(data_dir + output_result_file):
    os.remove(data_dir + output_result_file)

if '.csv' in data_file:
    df.coalesce(1).write.option("header", "true").csv(data_dir + output_result_file)
elif '.parquet' in data_file:
    df.coalesce(1).write.parquet(data_dir + output_result_file)
else:
    raise Exception('Neither .csv nor .parquet in file name')

file = glob.glob(data_dir + output_result_file + '/part-*')
shutil.move(file[0], data_dir + output_result_file + '.tmp')
shutil.rmtree(data_dir + output_result_file)
shutil.move(data_dir + output_result_file + '.tmp', data_dir + output_result_file)